# Notebook Description

This notebook is designed to create training data using grids from files and user-defined bounding boxes (list of (W, S, E, N) tuples).

## Processes:

1. **Read and Convert Predictors:**
   - Open each selected predictor file and convert it into a pandas Series.
   - Append these Series to a pandas DataFrame.

2. **Read EMAG Grid CSV:**
   - Open EMAG grid data from a CSV file and convert it into a pandas DataFrame. Join target and predictor dataframes.

3. **Define Boundary Boxes:**
   - Use the Shapely library to calculate and define the boundary boxes based on (W, S, E, N) tuples.

4. **Filter by Boundary Boxes:**
   - Apply the defined boundary boxes to filter out rows from the joined predictors and EMAG DataFrame.

## Implementation Details:

- **File Handling:** Uses Python's file handling capabilities to open and process predictor and EMAG grid files.
- **Data Conversion:** Converts predictor files into pandas Series and EMAG grid files into pandas DataFrame for easy manipulation.
- **Boundary Box Handling:** Implements boundary box calculations using Shapely library for spatial filtering.
- **Integration:** Integrates predictor data with EMAG data through DataFrame operations.
- **Data Filtering:** Filters data based on defined spatial boundaries to create refined training data.



* Define boundary boxes
* Apply filter_by_boundary_boxes(joined dataframes, boundary boxes)



# Utilities


In [20]:
pip install swifter

In [21]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import swifter
import os

# Define the boundary boxes for selection
boundary_boxes = [
    # USA: (latitude range: 24N to 49N, longitude range: 125W to 66W)
    (24, -125, 49, -66),    # Box 1: from (24N, 125W) to (49N, 66W)

    # Brazil: (latitude range: 5N to 34S, longitude range: 74W to 35W)
    (5, -74, -34, -35),     # Box 2: from (5N, 74W) to (34S, 35W)

    # Australia: (latitude range: 10S to 44S, longitude range: 113E to 154E)
    (-10, 113, -44, 154),   # Box 3: from (10S, 113E) to (44S, 154E)

    # Canada: (latitude range: 49N to 83N, longitude range: 141W to 52W)
    (49, -141, 83, -52),    # Box 4: from (49N, 141W) to (83N, 52W)

    # Argentina: (latitude range: 22S to 55S, longitude range: 73W to 53W)
    (-22, -73, -55, -53),   # Box 5: from (22S, 73W) to (55S, 53W)

    # China: (latitude range: 18N to 53N, longitude range: 74E to 135E)
    (18, 74, 53, 135),      # Box 6: from (18N, 74E) to (53N, 135E)

    # Russia: (latitude range: 41N to 82N, longitude range: 30E to 180E)
    (41, 30, 82, 180),      # Box 7: from (41N, 30E) to (82N, 180E)

    # South Africa: (latitude range: 22S to 35S, longitude range: 16E to 33E)
    (-22, 16, -35, 33),     # Box 8: from (22S, 16E) to (35S, 33E)

    # India: (latitude range: 8N to 37N, longitude range: 68E to 97E)
    (8, 68, 37, 97),        # Box 9: from (8N, 68E) to (37N, 97E)

    # Greenland: (latitude range: 60N to 84N, longitude range: 20W to 75W)
    (60, -75, 84, -20)      # Box 10: from (60N, 75W) to (84N, 20W)
]




In [22]:
# Function to check if a point is within any boundary box
def is_within_boundary_box(coords, box):
    min_latitude, min_longitude, max_latitude, max_longitude = box
    if (min_longitude < coords[1] < max_longitude and min_latitude < coords[0] < max_latitude):
      return True
    return False


# Function to check if a point is within any boundary box
def is_within_boundary_boxes(coords, boundary_boxes):
    for box in boundary_boxes:
        min_latitude, min_longitude, max_latitude, max_longitude = box
        if is_within_boundary_box(coords, box):
            return True
    return False

# Function to filter DataFrame based on boundary boxes
def filter_by_boundary_boxes(df, boundary_boxes):
    filtered_df = df[df.swifter.apply(lambda row: is_within_boundary_boxes((row['Latitude'], row['Longitude']), boundary_boxes), axis=1)]
    return filtered_df






def create_benchmark_grid(file, boundary_boxes):
  emag_ds = pd.read_csv('/content/EMAG2v3onPredictorMesh.csv')

  for file in os.listdir('/content/'):
    if file.endswith('.nc'):
      print(file)
      predictor_xarray = xr.open_dataset(f'/content/{file}')

      try:
        predictor_data = np.array(predictor_xarray['z']).flatten()
      except:
        print('error')
      predictor_ds = pd.DataFrame({file[:-10]: predictor_data})
      emag_ds = emag_ds.join(predictor_ds)

  # Save the filtered DataFrame to a CSV file
  filtered_emag = filter_by_boundary_boxes(emag_ds, boundary_boxes)
  return filtered_emag

# Grid Selection
Due to the size of our datasets, the model will be trained on several subsets of the globe. This notebook allows for the selection of training grids from csv file into a new csv file.

select_grid(file,include_list,boundary_boxes) = returns a dataframe of samples
* file = string of filepath
* include_list = list of feature name strings to include in dataset
* boundary_boxes = list of tuples (min lat, min lon, max lat, max lon)



In [14]:
atlantic_region = [(32, -58, 42, -48)]

holes = [
    (39,-56,40,-55),
    (34,-53,35,-52),
    (37,-50,38,-49)
]

boundary_box_1 = atlantic_region
boundary_box_2 = holes

df_1 = create_benchmark_grid('/content/EMAG2v3onPredictorMesh.csv',boundary_box_1)
df_2 = create_benchmark_grid('/content/EMAG2v3onPredictorMesh.csv',boundary_box_2)



3_gl_tot_sed_thick_100km^2.nc
14_wgm2012_freeair_ponc_100km^2.nc
10_sc_crust_vs_100km^2.nc
16_sl_vgg_eot_100km^2.nc
12_love_phase_100km^2.nc
4_cm_curie_point_depth_100km^2.nc
1_interpolated_mf7_100km^2.nc
2_interpolated_emm_100km^2.nc
6_interpolated_bouguer_100km^2.nc
18_igrf_inc_100km^2.nc
9__igrf_dec_100km^2.nc
15_rayleigh_phase_100km^2.nc
5_gl_elevation_100km^2.nc
7_sc_crust_vp_100km^2.nc
11_love_group_100km^2.nc
8_sc_crust_den_100km^2.nc
13_rayleigh_group_100km^2.nc
17_sc_crust_age_100km^2.nc


Dask Apply:   0%|          | 0/4 [00:00<?, ?it/s]

3_gl_tot_sed_thick_100km^2.nc
14_wgm2012_freeair_ponc_100km^2.nc
10_sc_crust_vs_100km^2.nc
16_sl_vgg_eot_100km^2.nc
12_love_phase_100km^2.nc
4_cm_curie_point_depth_100km^2.nc
1_interpolated_mf7_100km^2.nc
2_interpolated_emm_100km^2.nc
6_interpolated_bouguer_100km^2.nc
18_igrf_inc_100km^2.nc
9__igrf_dec_100km^2.nc
15_rayleigh_phase_100km^2.nc
5_gl_elevation_100km^2.nc
7_sc_crust_vp_100km^2.nc
11_love_group_100km^2.nc
8_sc_crust_den_100km^2.nc
13_rayleigh_group_100km^2.nc
17_sc_crust_age_100km^2.nc


Dask Apply:   0%|          | 0/4 [00:00<?, ?it/s]

# Outside Test Set Creation


Saving the holes to test on

In [15]:
save_df = df_2
filepath = input('Enter filepath: ')
save_df.to_csv(f'{filepath}.csv', index=False)

Enter filepath: /content/atlantic_region_testing_holes


creating holed training set

In [16]:
# Function to filter DataFrame based on boundary boxes
def filter_out_boundary_boxes(df, boundary_boxes):
    filtered_df = df[df.swifter.apply(lambda row: not is_within_boundary_boxes((row['Latitude'], row['Longitude']), boundary_boxes), axis=1)]
    return filtered_df


holed_df = filter_out_boundary_boxes(df_1, holes)

holed_df.to_csv('atlantic_region_hole_benchmark.csv', index=False)


Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

In [17]:
holed_df

,Longitude,Latitude,EMAG2v3,3_gl_tot_sed_thick_,14_wgm2012_freeair_ponc_,10_sc_crust_vs_,16_sl_vgg_eot_,12_love_phase_,4_cm_curie_point_depth_,1_interpolated_mf7_,...,6_interpolated_bouguer_,18_igrf_inc_,9__igrf_dec_,15_rayleigh_phase_,5_gl_elevation_,7_sc_crust_vp_,11_love_group_,8_sc_crust_den_,13_rayleigh_group_,17_sc_crust_age_
4546271,-57.936,32.068492,-2.564787,523.064636,-22.707346,3837.977539,-23.760847,4.750689,19.099236,45.434661,...,501.560498,58.011596,-17.618473,4.146347,-5213.958008,6757.968262,4.525249,2955.995605,3.902049,98.662010
4546272,-57.840,32.068492,13.700677,507.252808,-24.518305,3837.980713,-25.393854,4.751122,19.176317,45.366399,...,509.512351,57.978082,-17.641192,4.146088,-5281.347168,6757.972656,4.526880,2955.995605,3.902116,98.289299
4546273,-57.744,32.068492,23.453905,495.505554,-24.003344,3837.974365,-24.582073,4.751819,19.224140,45.133668,...,520.278083,57.944500,-17.663597,4.145797,-5312.617676,6757.963379,4.529567,2955.995850,3.902454,97.731895
4546274,-57.648,32.068492,26.664568,492.570129,-30.689665,3837.965088,-31.178421,4.752552,19.161064,44.749955,...,515.364111,57.910852,-17.685689,4.145618,-5565.387207,6757.948730,4.532524,2955.995361,3.903072,97.112625
4546275,-57.552,32.068492,24.131015,490.693726,-29.574699,3837.942627,-29.903376,4.753061,19.041849,44.226362,...,515.224138,57.877139,-17.707469,4.145586,-5560.247070,6757.913574,4.534694,2955.982910,3.903700,96.699181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4955120,-48.432,41.943173,-241.088839,3973.776611,-28.293646,3833.982422,-31.918360,4.772875,25.655724,-111.416869,...,367.134973,63.569893,-20.708443,4.174138,-3458.998779,6748.047363,4.539432,2950.224365,3.908380,NaN
4955121,-48.336,41.943173,-285.465974,4184.739746,-34.615894,3833.984131,-39.965725,4.772115,25.464323,-108.920762,...,366.163357,63.543780,-20.694595,4.172467,-3575.308105,6748.054688,4.539356,2950.228516,3.904670,NaN
4955122,-48.240,41.943173,-350.724173,4188.313477,-37.703247,3834.012207,-40.168747,4.771255,25.283077,-106.387305,...,367.959276,63.517698,-20.680521,4.170608,-3675.212158,6748.139648,4.539790,2950.276123,3.900704,NaN
4955123,-48.144,41.943173,-414.377670,3878.971436,-35.384350,3834.023193,-36.719975,4.770617,25.176542,-103.841412,...,373.750093,63.491645,-20.666222,4.169304,-3770.702393,6748.180176,4.540328,2950.298584,3.898047,NaN


In [18]:
print(df_1.count()[0] - df_2.count()[0] == holed_df.count()[0] )

True


# Plotting Grid Selections
Plots all selected boundary boxes for each variable in dataset. Results in (variables * boxes) # of plots. This is used to visually validate that we are taking data from the area, and to analyze spaital correlation intuitively.


In [23]:
def plot_on_selections(filtered_emag, boundary_boxes):
  for column in filtered_emag.columns:
      if column != 'Longitude' and column != 'Latitude':
        for box in boundary_boxes:
          plotting_emag = filter_by_boundary_boxes(filtered_emag, boundary_boxes)
          plotting_emag = plotting_emag.reset_index(drop=True)

          grid_df = plotting_emag.pivot(index='Latitude', columns='Longitude', values = column )

          # Step 3: Convert the pivoted DataFrame to an xarray.DataArray
          data_array = xr.DataArray(grid_df.values,
                                    coords=[grid_df.index, grid_df.columns],
                                    dims=['Latitude', 'Longitude'],
                                    name=column)


          try:
            data_array.plot()
            plt.title(f'{column} Grid Data')
            plt.xlabel('Longitude')
            plt.ylabel('Latitude')
            plt.show()
          except:
            continue
plot_on_selections(df_1, boundary_boxes)

Dask Apply:   0%|          | 0/4 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11440 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 